In [1]:
import numpy as np
import tensorflow as tf

In [2]:
sentences = [['hi'],
             ['hello'],
             ['hi', 'there', 'how', 'are', 'you'],
             ['i', 'am', 'fine', 'how', 'do', 'you', 'do'],
             ['i', 'am', 'doing', 'fine'],
             ['what', 'is', 'your', 'name'],
             ['hi', 'my', 'name', 'is', 'strange'],
             ['that', 'is', 'indeed', 'strange'],
             ['hello', 'darkness', 'my', 'old', 'friend'],
             ['darkness', 'is', 'strange']]

In [3]:
person1 = []
person2 = []
for i in range(len(sentences)):
    if i%2 == 0:
        person1.append(sentences[i])
    else:
        person2.append(sentences[i])

In [4]:
person1

[['hi'],
 ['hi', 'there', 'how', 'are', 'you'],
 ['i', 'am', 'doing', 'fine'],
 ['hi', 'my', 'name', 'is', 'strange'],
 ['hello', 'darkness', 'my', 'old', 'friend']]

In [5]:
person2

[['hello'],
 ['i', 'am', 'fine', 'how', 'do', 'you', 'do'],
 ['what', 'is', 'your', 'name'],
 ['that', 'is', 'indeed', 'strange'],
 ['darkness', 'is', 'strange']]

In [6]:
words = []
[[words.append(w) for w in sent] for sent in sentences];
vocab = list(set(words))
vocab_size = len(vocab)

In [7]:
vocab_size

22

In [8]:
id_to_word = dict(enumerate(vocab))
word_to_id = {v:k for k,v in id_to_word.items()}

In [9]:
id_to_word[22] = ''

In [10]:
seq_len = max([len(s) for s in sentences])

In [11]:
seq_len

7

In [12]:
for i in range(len(person1)):
    person1[i] = [word_to_id[w] for w in person1[i]] + [vocab_size] * (seq_len - len(person1[i]))
    person2[i] = [word_to_id[w] for w in person2[i]] + [vocab_size] * (seq_len - len(person2[i]))

In [13]:
person1 = np.array(person1)

In [14]:
person2 = np.array(person2)

In [15]:
vec = np.zeros((vocab_size+1, vocab_size+1), dtype=np.float64)
for i in range(vocab_size+1):
    vec[i,i] = 1.0

In [16]:
input_shape = output_shape = vocab_size+1
hidden_shape = 64
learning_rate = 0.001
batch_size = person1.shape[0]

In [17]:
word_to_id

{'am': 16,
 'are': 17,
 'darkness': 0,
 'do': 15,
 'doing': 13,
 'fine': 1,
 'friend': 3,
 'hello': 12,
 'hi': 8,
 'how': 10,
 'i': 14,
 'indeed': 2,
 'is': 6,
 'my': 5,
 'name': 4,
 'old': 9,
 'strange': 19,
 'that': 11,
 'there': 18,
 'what': 7,
 'you': 21,
 'your': 20}

In [18]:
a = np.array([[8, 10, 17, 21, 22, 22, 22]])
[id_to_word[i] for i in a[0]]

['hi', 'how', 'are', 'you', '', '', '']

In [35]:
tf.reset_default_graph()
with tf.Graph().as_default() as graph:
    
    X = tf.placeholder(shape=[None,seq_len], dtype=tf.int32, name="input")
    Y = tf.placeholder(shape=[None,seq_len], dtype=tf.int32, name="target")
    embed = tf.constant(vec, name="embeddings", dtype=tf.float64)
    X_embed = tf.nn.embedding_lookup(embed, X, name="input_embeddings")
    X_embed = tf.transpose(X_embed, perm=[1,0,2])
    Y_embed = tf.nn.embedding_lookup(embed, Y, name="input_embeddings")
    h_in = tf.placeholder(shape=[None, hidden_shape], dtype=tf.float64)
    
    def RNN(x_t, h_prev):
        with tf.variable_scope('RNN'):
            
            W_xh = tf.get_variable(name="W_xh", shape=[input_shape, hidden_shape], 
                                   initializer=tf.random_normal_initializer(mean=0.0, 
                                                                            stddev=0.1), 
                                   dtype=tf.float64)
            
            W_hh = tf.get_variable(name="W_hh", shape=[hidden_shape, hidden_shape], 
                                   initializer=tf.random_normal_initializer(mean=0.0, 
                                                                            stddev=0.1), 
                                   dtype=tf.float64)
            
            W_yh = tf.get_variable(name="W_yh", shape=[hidden_shape, output_shape], 
                                   initializer=tf.random_normal_initializer(mean=0.0, 
                                                                            stddev=0.1), 
                                   dtype=tf.float64)
            
            h_t = tf.tanh(tf.matmul(x_t, W_xh) + tf.matmul(h_prev, W_hh))
            
            y_t = tf.nn.softmax(tf.matmul(h_t, W_yh), name="output")
            
            h_t = tf.reshape(h_t, shape=[-1, hidden_shape])
            
            return [h_t, y_t]
    
    def encoder_helper(h_prev, x_t):
        with tf.variable_scope("encoder_helper"):
            
            h_t,_ = RNN(x_t, h_prev)
            
            return h_t
    
    def encoder(inputs, h_in):
        with tf.variable_scope("encoder"):
            
            out_encoder = tf.scan(encoder_helper, inputs, initializer=h_in, name='encoder_output')
            
            return out_encoder[-1]
    
    def decoder_helper(inputs, x_t):
        with tf.variable_scope("decoder_helper"):
                        
            h_prev, y_prev = inputs[0], inputs[1]
            
            y_prev = tf.reshape(y_prev, [-1, input_shape])
            
            h_prev = tf.reshape(h_prev, shape=[-1, hidden_shape])
            
            h_t, y_t = RNN(y_prev, h_prev)

            outputs = [h_t, y_t]
            
            return outputs
    
    def decoder(inputs, h_in):
        with tf.variable_scope('decoder'):
            
            out_decoder = tf.scan(decoder_helper, X_embed, initializer=[inputs, h_in])
            
            return out_decoder[1]
    
    encoder_output = encoder(X_embed, h_in)
    
    W_y = tf.get_variable(name="W_y", shape=[hidden_shape, output_shape], 
                          initializer=tf.random_normal_initializer(mean=0.0, 
                                                                   stddev=0.1), 
                          dtype=tf.float64)
    
    dec_in = tf.nn.softmax(tf.matmul(encoder_output, W_y), name='decoder_input')
    
    decoder_output = decoder(encoder_output, dec_in)
    
    seq_output = tf.transpose(decoder_output, perm=[1, 0, 2])
    
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_embed, 
                                                                      logits=seq_output))
    
    with tf.name_scope('train'):
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    
    output = tf.argmax(decoder_output, axis=2)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        
        for i in range(10001):
            
            _,l = sess.run([optimizer, loss], 
                           {X:person1, 
                            Y:person2, 
                            h_in: np.zeros((batch_size, hidden_shape))})
            
            if i%1000 == 0:
                print(l)
            
            if i%5000 == 0:
                pred = sess.run(output, {X:person1, Y:person2, h_in: np.zeros((batch_size, hidden_shape))})
                for j in range(batch_size):
                    print("person1 : ", [[id_to_word[w] for w in sent] for sent in person1][j])
                    print("person2 : ", [[id_to_word[w] for w in sent] for sent in pred.T][j])
                    print("====================================================")
                    
        
        out = sess.run(output, {X:a, Y:person2, h_in: np.zeros((1, hidden_shape))})
        print("personA : ", [[id_to_word[w] for w in sent] for sent in a])
        print("bot     : ", [[id_to_word[w] for w in sent] for sent in out.T])
        writer = tf.summary.FileWriter('tmp/1')
        writer.add_graph(sess.graph)

3.13503145452
person1 :  ['hi', '', '', '', '', '', '']
person2 :  ['', 'that', 'indeed', 'strange', '', '', '']
person1 :  ['hi', 'there', 'how', 'are', 'you', '', '']
person2 :  ['', 'that', 'there', 'strange', 'is', '', '']
person1 :  ['i', 'am', 'doing', 'fine', '', '', '']
person2 :  ['', 'is', 'there', 'strange', 'is', '', '']
person1 :  ['hi', 'my', 'name', 'is', 'strange', '', '']
person2 :  ['fine', 'that', 'fine', 'strange', '', '', '']
person1 :  ['hello', 'darkness', 'my', 'old', 'friend', '', '']
person2 :  ['fine', 'that', 'there', 'my', 'you', '', '']
2.23610046142
2.20771620463


KeyboardInterrupt: 

In [20]:
! tensorboard --logdir ./tmp/1

Starting TensorBoard b'41' on port 6006
(You can navigate to http://127.0.1.1:6006)
^CTraceback (most recent call last):
  File "/home/shivam/anaconda3/bin/tensorboard", line 11, in <module>
    sys.exit(main())
  File "/home/shivam/anaconda3/lib/python3.5/site-packages/tensorflow/tensorboard/tensorboard.py", line 151, in main
    tb_server.serve_forever()
  File "/home/shivam/anaconda3/lib/python3.5/socketserver.py", line 232, in serve_forever
    ready = selector.select(poll_interval)
  File "/home/shivam/anaconda3/lib/python3.5/selectors.py", line 376, in select
    fd_event_list = self._poll.poll(timeout)
KeyboardInterrupt

